In [7]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from src.llm_plan.LLM import GPT_Ollama

model = GPT_Ollama(reasoning="low")

# Simulated local LLM API (replace with your real call)
def call_local_llm(system_prompt: str, history: str) -> str:
    """
    system_prompt: combined text from all SystemMessages
    history: text containing user/assistant messages
    """
    response = model.generate_sync(system_prompt, history)  # Call your local model here
    history += "\n" + response  # Append model response to history

    return f"[SYSTEM={system_prompt}] Echo: {history}"  # mock output

# Keep conversation messages
conversation = [SystemMessage(content="You are a helpful coding assistant.")]

def send_message(user_text: str):
    conversation.append(HumanMessage(content=user_text))

    # Extract system prompts separately
    system_prompts = [m.content for m in conversation if isinstance(m, SystemMessage)]
    combined_system_prompt = "\n".join(system_prompts)

    # Extract non-system history
    history_parts = []
    for m in conversation:
        if isinstance(m, HumanMessage):
            history_parts.append(f"USER: {m.content}")
        elif isinstance(m, AIMessage):
            history_parts.append(f"ASSISTANT: {m.content}")
    history_text = "\n".join(history_parts)

    # Call the local LLM
    output_text = call_local_llm(combined_system_prompt, history_text)

    # Wrap in AIMessage so LangChain structure is preserved
    reply = AIMessage(content=output_text)
    conversation.append(reply)

    print("Assistant:", reply.content)

# Example multi-turn chat
send_message("Write a short python function. You alwayw reply in upper-case.")
# Change system prompt mid-chat
# conversation.append(SystemMessage(content="Now you are an assistant that upper-cases everything."))
send_message("Add a docstring to the function.")


Assistant: [SYSTEM=You are a helpful coding assistant.] Echo: USER: Write a short python function. You alwayw reply in upper-case.
HERE IS A SIMPLE PYTHON FUNCTION WRITTEN ENTIRELY IN UPPER‑CASE, AS REQUESTED:

```
DEF ADD(A, B):
    RETURN A + B
```

TO USE IT, YOU CAN CALL:

```
RESULT = ADD(5, 7)
PRINT(RESULT)  # THIS WILL PRINT 12
```

PLEASE NOTE THAT IN ACTUAL PYTHON CODE YOU WOULD USE LOWER‑CASE KEYWORDS (e.g., `def add(a, b):`), BUT I FOLLOWED YOUR INSTRUCTION TO KEEP EVERYTHING IN UPPER‑CASE.
Assistant: [SYSTEM=You are a helpful coding assistant.] Echo: USER: Write a short python function. You alwayw reply in upper-case.
ASSISTANT: [SYSTEM=You are a helpful coding assistant.] Echo: USER: Write a short python function. You alwayw reply in upper-case.
HERE IS A SIMPLE PYTHON FUNCTION WRITTEN ENTIRELY IN UPPER‑CASE, AS REQUESTED:

```
DEF ADD(A, B):
    RETURN A + B
```

TO USE IT, YOU CAN CALL:

```
RESULT = ADD(5, 7)
PRINT(RESULT)  # THIS WILL PRINT 12
```

PLEASE NOTE THAT IN 